In [1]:
# Import all the necessary modules
from dagrad import dagrad # dagrad is the main class for learning the structure of a DAG
from dagrad import generate_linear_data, generate_nonlinear_data, count_accuracy

### Linear Model - (N)on-equal variances

In [ ]:
# ER1 graph
n, d, s0, graph_type, sem_type = 1000, 10, 10, 'ER', 'gauss' # Define the parameters of the data
X, W_true, B_true = generate_linear_data(n,d,s0,graph_type,sem_type,seed  =2) # Generate the data
model = 'linear' # Define the model
W_golem = dagrad(X, model = model, method = 'dagma') # Learn the structure of the DAG using Golem
print(f"Linear Model")
print(f"data size: {n}, graph type: {graph_type}, sem type: {sem_type}")

acc_golem = count_accuracy(B_true, W_golem != 0) # Measure the accuracy of the learned structure using Golem
print('Accuracy of Golem:', acc_golem)
